<a href="https://colab.research.google.com/github/anage-wq/XIRR_PIPELINE/blob/main/Pipeline_XIRR_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

In [6]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

Inputs:

1.Trade Book

2.Start Date and End Date

3.Stock split



Trade Book


In [7]:
tradebook=pd.read_csv('/content/TradeBook.csv')


Dates


In [8]:
import json
import pandas as pd

with open('/content/Date_Input .json', 'r') as f:
    date_data = json.load(f)

start_date = pd.to_datetime(date_data['start_date'], dayfirst=True)




In [9]:
start_date

Timestamp('2023-06-01 00:00:00')

Stock split extra


In [10]:
import json
import pandas as pd
with open('/content/Stock_split_external.json', 'r') as f:
    stock_split_extra = json.load(f)





In [11]:
stock_split_extra


{'Company_Ticker': [{'Date': 'Stock_split'}]}

In [12]:
for i,j in stock_split_extra.items():
  print(i)
  print(j[0])

Company_Ticker
{'Date': 'Stock_split'}


In [13]:
records = []
# Check if 'Company_Ticker' exists and is a list
if 'Company_Ticker' in stock_split_extra and isinstance(stock_split_extra['Company_Ticker'], list):
    for item in stock_split_extra['Company_Ticker']:
        # Check if the item is a dictionary before using .get()
        if isinstance(item, dict):
            symbol = item.get('symbol')
            date = item.get('date')
            stock_split = item.get('stock_split')
            records.append({'symbols': symbol, 'dates': date, 'stock_split': stock_split})
        else:
            print(f"Warning: Expected a dictionary in the 'Company_Ticker' list but got {type(item)}")

stock_split_extra_df = pd.DataFrame(records, columns=['symbols', 'dates', 'stock_split'])

In [14]:
stock_split_extra_df

,symbols,dates,stock_split
0,None,None,None


XIRR ANALYSIS

1st transformer


In [15]:
def preprocessing(ds):

  ds = ds.drop(['Segment','Series','Auction','Trade ID','Order ID'],axis=1) # Direct assignment after drop
  ds['Trade Type'] = ds['Trade Type'].str.lower() # Vectorized lower case
  from datetime import datetime
  ds['Trade Date'] = pd.to_datetime(ds['Trade Date'], dayfirst=True, errors='coerce', format='mixed')
  ds['Quantity'] = ds['Quantity'].astype(str).str.replace(',', '', regex=False)
  ds['Quantity'] = pd.to_numeric(ds['Quantity'], errors='coerce')
  ds['Price'] = ds['Price'].astype(str).str.replace(',', '', regex=False)
  ds['Price'] = pd.to_numeric(ds['Price'], errors='coerce')

  return ds

In [16]:
trf1=FunctionTransformer(preprocessing )

In [17]:
ds1=preprocessing(tradebook)

In [18]:
ds1

,Symbol,ISIN,Trade Date,Exchange,Trade Type,Quantity,Price,Order Execution Time
0,APOLLOHOSP,INE437A01024,2023-05-22,NSE,buy,4,4584.350000,2023-05-22T14:00:01
1,PERSISTENT,INE262H01013,2023-05-22,NSE,buy,4,4965.950000,2023-05-22T14:00:01
2,KPITTECH,INE04I401011,2023-05-22,NSE,buy,13,937.850000,2023-05-22T14:00:01
3,KPITTECH,INE04I401011,2023-05-22,NSE,buy,7,937.900000,2023-05-22T14:00:01
4,MAXHEALTH,INE027H01010,2023-05-22,NSE,buy,1,550.450000,2023-05-22T14:00:01
...,...,...,...,...,...,...,...,...
4811,FIEMIND,NaN,2025-10-08,NSE,buy,17,1923.394118,2025-10-08 12:20:09
4812,REDINGTON,NaN,2025-10-08,NSE,buy,120,276.850000,2025-10-08 12:20:10
4813,SARDAEN,NaN,2025-10-08,NSE,buy,58,569.125000,2025-10-08 12:20:12
4814,TI,NaN,2025-10-08,NSE,buy,71,468.681690,2025-10-08 12:20:13


2nd Transformer


In [28]:
def second(ds,stock_split_data_ext):
  #1 conversion of quantity and price
  ds['Quantity']=ds['Quantity'].astype(float)
  ds['Price']=ds['Price'].astype(float)
  #2 symbol,isin,exchange data
  comp_info=tradebook[['Symbol','ISIN','Exchange']]
  comp_info_unique = comp_info.drop_duplicates(subset=['Symbol'], keep='first')
  comp_info_unique.reset_index(drop=True, inplace=True)


  #3company ambiguity
  import yfinance as yf


  isin_symbol_counts = comp_info_unique.groupby('ISIN')['Symbol'].nunique()

# Filter for ISINs that have more than one unique symbol
  isin_with_multiple_symbols = isin_symbol_counts[isin_symbol_counts > 1].index

  # Create a new DataFrame containing only the rows where the ISIN is in the list of ISINs with multiple symbols
  symbols_with_repeated_isin = comp_info_unique[comp_info_unique['ISIN'].isin(isin_with_multiple_symbols)]

  #delisting function
  def is_delisted(symbol, exchange_suffix='.NS'):
      """
      Returns True if the symbol is likely delisted or unavailable on Yahoo Finance, False otherwise.
      """
      try:
          ticker = yf.Ticker(symbol + exchange_suffix)
          hist = ticker.history(period='1d')
          # If no historical data, treat as delisted or not available
          return hist.empty
      except Exception:
          return True

  #to chnge and chnge to


  to_chnge=[]
  chang_to=[]





  for i in range(symbols_with_repeated_isin.shape[0]):
    sym=symbols_with_repeated_isin['Symbol'].iloc[i]

    if symbols_with_repeated_isin['Exchange'].iloc[i]=='NSE':
      suf='.NS'
    if symbols_with_repeated_isin['Exchange'].iloc[i]=='BSE':
      suf='.BO'

    if is_delisted(sym,suf)==False:
      chang_to.append(sym)
    if is_delisted(sym,suf)==True:
      to_chnge.append(sym)


  for i in range(tradebook.shape[0]):
    if tradebook['Symbol'].iloc[i] in to_chnge:
      ind = to_chnge.index(tradebook['Symbol'].iloc[i])
      tradebook.loc[i, 'Symbol'] = chang_to[ind]
      # Find the index in comp_info_unique for the changed symbol
      indx = comp_info_unique.index[comp_info_unique['Symbol'] == chang_to[ind]].tolist()
      if indx: # Check if index is found
          tradebook.loc[i, 'Exchange'] = comp_info_unique['Exchange'].iloc[indx[0]]

  ds_help=tradebook.copy()
  filtered_comps_isin_exchange=tradebook[['Symbol','ISIN','Exchange']]
  filtered_comps_isin_exchange=filtered_comps_isin_exchange.drop_duplicates(subset=['Symbol'], keep='first')
  filtered_comps_isin_exchange


  stock_split_data_dict={}
  for i in range(filtered_comps_isin_exchange.shape[0]):
    sym=filtered_comps_isin_exchange['Symbol'].iloc[i]
    if filtered_comps_isin_exchange['Exchange'].iloc[i]=='NSE':
      suf='.NS'
    if filtered_comps_isin_exchange['Exchange'].iloc[i]=='BSE':
      suf='.BO'

    ticker=yf.Ticker(sym+suf)
    splits=ticker.splits
    splits = splits.reset_index()
    stock_split_data_dict[sym]=splits




  for i,j in stock_split_data_ext.items():
    # Assuming stock_split_data_ext is a dictionary of dataframes where keys are symbols
    if isinstance(j, pd.DataFrame):
        stock_split_data_dict[i] = j
    else:
        print(f"Warning: External data for symbol {i} is not a DataFrame. Skipping.")




  #adjustment
  for symbol, split_info in stock_split_data_dict.items():
      if not isinstance(split_info, pd.DataFrame) or split_info.empty:
          print(f"Skipping split adjustment for symbol {symbol} due to invalid split info.")
          continue

      # Determine column names based on source (assuming 'Date' is always the date column)
      if 'Stock Splits' in split_info.columns: # Assuming yfinance data has 'Stock Splits'
          split_date_col = 'Date'
          split_factor_col = 'Stock Splits'
      elif 'stock_split' in split_info.columns: # Assuming external data has 'stock_split'
          split_date_col = 'Date'
          split_factor_col = 'stock_split'
      else:
          print(f"Warning: Could not find split date or factor columns for symbol {symbol}. Skipping.")
          continue


      for index, row in split_info.iterrows():
          try:
              split_date = pd.to_datetime(row[split_date_col])
              # Make the split_date timezone-naive for comparison
              if split_date.tzinfo is not None:
                  split_date = split_date.tz_convert(None)

              split_factor = float(row[split_factor_col]) # Ensure split factor is float

              # Apply adjustment only to trades before the split date
              mask = (ds['Symbol'] == symbol) & (ds['Trade Date'] < split_date)
              ds.loc[mask, 'Quantity'] *= split_factor
              ds.loc[mask, 'Price'] /= split_factor
          except (KeyError, ValueError, TypeError) as e: # Added TypeError to catch potential conversion issues
              print(f"Error processing split for symbol {symbol} on date {row.get(split_date_col, 'N/A')}: {e}")
              continue


  return ds

In [30]:
ds2

,Symbol,ISIN,Trade Date,Exchange,Trade Type,Quantity,Price,Order Execution Time
0,APOLLOHOSP,INE437A01024,2023-05-22,NSE,buy,4.0,4584.350000,2023-05-22T14:00:01
1,PERSISTENT,INE262H01013,2023-05-22,NSE,buy,8.0,2482.975000,2023-05-22T14:00:01
2,KPITTECH,INE04I401011,2023-05-22,NSE,buy,13.0,937.850000,2023-05-22T14:00:01
3,KPITTECH,INE04I401011,2023-05-22,NSE,buy,7.0,937.900000,2023-05-22T14:00:01
4,MAXHEALTH,INE027H01010,2023-05-22,NSE,buy,1.0,550.450000,2023-05-22T14:00:01
...,...,...,...,...,...,...,...,...
4811,FIEMIND,NaN,2025-10-08,NSE,buy,17.0,1923.394118,2025-10-08 12:20:09
4812,REDINGTON,NaN,2025-10-08,NSE,buy,120.0,276.850000,2025-10-08 12:20:10
4813,SARDAEN,NaN,2025-10-08,NSE,buy,58.0,569.125000,2025-10-08 12:20:12
4814,TI,NaN,2025-10-08,NSE,buy,71.0,468.681690,2025-10-08 12:20:13


In [ ]:
trf2=FunctionTransformer(second)

3.cashflow transformer


In [31]:
def cashFlow(ds):
  ds['Cash_Flow'] = ds.apply(
    lambda row: -(row['Quantity'] * row['Price']) if str(row['Trade Type'])== 'buy'
                else (row['Quantity'] * row['Price']),
    axis=1
  )
  return ds

In [ ]:
trf3=FunctionTransformer(cashFlow)

4.XIRR transformer


In [36]:
def calculate_xirr_with_current_prices(tradebook_df):
    from scipy.optimize import newton
    from datetime import date
    import pandas as pd
    import yfinance as yf
    import pytz

    # --- Internal helper functions for NPV and XIRR ---
    def _npv(rate, dates, cashflows):
        """Calculate Net Present Value."""
        # Ensure dates are in datetime format and timezone-naive
        dates = [pd.to_datetime(d).tz_convert(None) if pd.to_datetime(d).tzinfo is not None else pd.to_datetime(d) for d in dates]

        # Calculate the time difference in years
        # Ensure dates are sorted before calculating time differences
        dates.sort()
        if not dates: # Handle case where dates list is empty
            return 0.0 # Or raise an error, depending on desired behavior

        time_in_years = [(d - dates[0]).days / 365.0 for d in dates]
        return sum(cf / (1 + rate) ** ti for cf, ti in zip(cashflows, time_in_years))


    def _xirr(dates, cashflows):
        """Calculate Extended Internal Rate of Return."""
        # Check if there are both positive and negative cash flows
        if not any(cf > 0 for cf in cashflows) or not any(cf < 0 for cf in cashflows):
            return None
        try:
            # Use Newton-Raphson method to find the root of the NPV equation.
            # Provide a range for the initial guess (x0) to help convergence
            # Adjust bounds as needed based on expected XIRR values
            return newton(lambda r: _npv(r, dates, cashflows), x0=0.1, tol=1e-6, maxiter=100)
        except RuntimeError:
            # Return None if solver does not converge.
            return None
        except ValueError:
            # Return None if input values are not appropriate (e.g., dates not unique)
            return None

    def get_current_stock_price(symbol: str) -> float:
        """
        Fetches the current market price of a stock from Yahoo Finance.

        Args:
            symbol (str): The stock ticker symbol (e.g., 'RELIANCE.NS', 'AAPL').

        Returns:
            float: The current market price, or None if not found.
        """
        try:
            stock = yf.Ticker(symbol)
            # The 'info' attribute contains a dictionary of stock information
            price = stock.info.get('regularMarketPrice')
            return price
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None


    # --- 1. Fetch current stock prices ---
    unique_symbols = tradebook_df['Symbol'].unique()
    current_prices = {}

    for symbol in unique_symbols:
        # Add the appropriate exchange suffix if not already present
        if '.' not in symbol:
            # This is a simplification; a more robust approach would determine the exchange from the data
            # For this example, we'll assume '.NS' for NSE
            # You might need to adapt this based on your data's structure and exchanges
            symbol_with_exchange = f"{symbol}.NS" # Default to .NS

        else:
            symbol_with_exchange = symbol

        price = get_current_stock_price(symbol_with_exchange)
        if price is not None:
            current_prices[symbol] = price

    # --- 2. Calculate the value of current holdings ---

    # Create a temporary column to calculate net holdings easily.
    df = tradebook_df.copy()
    # Ensure column names are consistent and use .loc for assignment
    df.loc[:, 'signed_quantity'] = df.apply(
        lambda row: row['Quantity'] if str(row['Trade Type']).lower() == 'buy'
                else -row['Quantity'],
        axis=1
    )

    # Group by symbol to find the net quantity of each stock held.
    holdings = df.groupby('Symbol')['signed_quantity'].sum()

    # Calculate the total market value of the stocks you still own.
    current_portfolio_value = 0
    for symbol, quantity in holdings.items():
        if quantity > 0:  # Only include stocks with a positive holding.
            price = current_prices.get(symbol)
            if price is None:
                 # Handle missing prices gracefully, perhaps skip or log a warning
                print(f"Warning: Current price for '{symbol}' not found. Skipping this holding.")
                continue # Skip this symbol if price is not available
            current_portfolio_value += quantity * price

    # --- 3. Prepare the final cash flow series ---

    # Get the original transaction cash flows and dates.
    # Ensure column names are consistent
    # Drop rows with missing values in 'Trade Date' or 'Cash_Flow' before converting to lists
    df_cleaned = df.dropna(subset=['Trade Date', 'Cash_Flow']).copy()
    dates = df_cleaned['Trade Date'].tolist()
    cashflows = df_cleaned['Cash_Flow'].tolist()


    # Add the current portfolio value as the final cash inflow event.
    # This represents the value you would get if you sold everything today.
    if current_portfolio_value > 0 and dates: # Also check if dates is not empty
        # Ensure today's date is timezone-naive
        today_naive = pd.to_datetime(date.today())
        dates.append(today_naive)
        cashflows.append(current_portfolio_value)

    # --- 4. Calculate and return XIRR ---
    # Ensure dates and cashflows are aligned and sorted by date for accurate XIRR
    # Combine dates and cashflows into a list of tuples and sort
    if not dates or not cashflows or len(dates) != len(cashflows):
        return None # Return None if cash flow data is not valid for XIRR calculation

    cashflow_events = sorted(zip(dates, cashflows))
    sorted_dates, sorted_cashflows = zip(*cashflow_events)

    xirr_value = _xirr(list(sorted_dates), list(sorted_cashflows))

    return xirr_value

In [ ]:
trf4=FunctionTransformer(calculate_xirr_with_current_prices)

PIPELINE


In [ ]:
pipe=Pipeline([
    ('first',trf1),
    ('second',trf2),
    ('third',trf3),
    ('fourth',trf4)

])

In [ ]:
ds=pipe.transform(fy_all)

ERROR:yfinance:$TIPSINDLTD.NS: possibly delisted; no timezone found
ERROR:yfinance:$TATAINVEST-BE.NS: possibly delisted; no timezone found
ERROR:yfinance:$HBLPOWER.NS: possibly delisted; no timezone found
ERROR:yfinance:$IIFLSEC.NS: possibly delisted; no timezone found
/tmp/ipython-input-1083555384.py:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 55.   55.   55.   15.    2.5  37.5  42.5  12.5 130.  130.    2.5   7.5
  90.    7.5  27.5  15.   47.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ds.loc[mask, 'Quantity'] = ds.loc[mask, 'Quantity'].astype(float) * split_ratio


In [ ]:
print("The calculated XIRR is:")
print(ds)

The calculated XIRR is:
0.49229832785730854


In [ ]:
trf3=FunctionTransformer(cashFlow)